In [1]:
datasets=["10k_pbmc","5k_pbmc",
          "pbmc_cite",
          "openproblems_2022_multi_atac2gex",
          "openproblems_2022_cite_gex2adt",
          "GSE127064_AdBrain_gex2atac",
          "GSE117089_sciCAR_gex2atac",
          "GSE127064_p0Brain_gex2atac",
          "GSE117089_mouse_gex2atac",
          "GSE140203_3T3_HG19_atac2gex",
        "GSE140203_3T3_MM10_atac2gex",
        "GSE140203_12878.rep2_atac2gex",
        "GSE140203_12878.rep3_atac2gex",
        "GSE140203_K562_HG19_atac2gex",
        "GSE140203_K562_MM10_atac2gex",
        "GSE140203_LUNG_atac2gex"]

In [2]:
train_leng=7000
test_leng=3000

In [3]:
import argparse
import logging
import os
import random

import torch

from dance import logger
from dance.datasets.multimodality import ModalityPredictionDataset
from dance.modules.multi_modality.predict_modality.babel import BabelWrapper
from dance.utils import set_seed


OPTIMIZER_DICT = {
    "adam": torch.optim.Adam,
    "rmsprop": torch.optim.RMSprop,
}
BabelWrapper_scores=[]
rndseed = random.randint(0, 2147483647)
parser = argparse.ArgumentParser()
parser.add_argument("-t", "--subtask", default="openproblems_bmmc_cite_phase2_rna")
parser.add_argument("-device", "--device", default="cuda")
parser.add_argument("-cpu", "--cpus", default=1, type=int)
parser.add_argument("-seed", "--rnd_seed", default=rndseed, type=int)
parser.add_argument("-m", "--model_folder", default="./models")
parser.add_argument("--outdir", "-o", default="./logs", help="Directory to output to")
parser.add_argument("--lossweight", type=float, default=1., help="Relative loss weight")
parser.add_argument("--lr", "-l", type=float, default=0.01, help="Learning rate")
parser.add_argument("--batchsize", "-b", type=int, default=64, help="Batch size")
parser.add_argument("--hidden", type=int, default=64, help="Hidden dimensions")
parser.add_argument("--earlystop", type=int, default=5, help="Early stopping after N epochs")
parser.add_argument("--naive", "-n", action="store_true", help="Use a naive model instead of lego model")
parser.add_argument("--resume", action="store_true")
parser.add_argument("--span", default=0.3, type=float)
parser.add_argument("--max_epochs", type=int, default=10)
for dataset in datasets:
    v=None
    try:
        args = parser.parse_args(['--subtask',dataset,'--device','cuda:1','--span','1.0'])
        args.resume = True

        torch.set_num_threads(args.cpus)
        rndseed = args.rnd_seed
        set_seed(rndseed)
        dataset = ModalityPredictionDataset(args.subtask, preprocess="feature_selection",span=args.span)
        data = dataset.load_data()

        device = args.device
        args.outdir = os.path.abspath(args.outdir)
        os.makedirs(args.model_folder, exist_ok=True)
        os.makedirs(args.outdir, exist_ok=True)

        # Specify output log file
        fh = logging.FileHandler(f"{args.outdir}/training_{args.subtask}_{args.rnd_seed}.log", "w")
        fh.setLevel(logging.INFO)
        logger.addHandler(fh)

        for arg in vars(args):
            logger.info(f"Parameter {arg}: {getattr(args, arg)}")

        # Obtain training and testing data
        x_train, y_train = data.get_train_data(return_type="torch")
        x_test, y_test = data.get_test_data(return_type="torch")
        x_train, y_train= x_train.float()[:train_leng], y_train.float()[:train_leng]
        x_test, y_test =x_test.float()[:test_leng], y_test.float()[:test_leng]
        # Train and evaluate the model
        model = BabelWrapper(args, dim_in=x_train.shape[1], dim_out=y_train.shape[1])
        model.fit(x_train, y_train, val_ratio=0.15)
        print(model.predict(x_test))
        score=model.score(x_test, y_test)
        
    except (Exception, BaseException) as e:
        v=e
    else:
        v=score
    finally:
        print(v)
        BabelWrapper_scores.append(v)
        torch.cuda.empty_cache()
"""To reproduce BABEL on other samples, please refer to command lines belows:
GEX to ADT (subset):
python babel.py --subtask openproblems_bmmc_cite_phase2_rna_subset --device cuda

GEX to ADT:
python babel.py --subtask openproblems_bmmc_cite_phase2_rna --device cuda

ADT to GEX:
python babel.py --subtask openproblems_bmmc_cite_phase2_mod2 --device cuda

GEX to ATAC:
python babel.py --subtask openproblems_bmmc_multiome_phase2_rna --device cuda

ATAC to GEX:
python babel.py --subtask openproblems_bmmc_multiome_phase2_mod2 --device cuda
"""


[INFO][2023-09-19 23:21:11,995][dance][set_seed] Setting global random seed to 2012473794
[INFO][2023-09-19 23:21:11,996][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/10k_pbmc/10k_pbmc.10kanti_dataset_subset.output_train_mod1.h5ad
[INFO][2023-09-19 23:21:12,059][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/10k_pbmc/10k_pbmc.10kanti_dataset_subset.output_train_mod2.h5ad
[INFO][2023-09-19 23:21:12,194][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/10k_pbmc/10k_pbmc.10kanti_dataset_subset.output_test_mod1.h5ad
[INFO][2023-09-19 23:21:12,240][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/10k_pbmc/10k_pbmc.10kanti_dataset_subset.output_test_mod2.h5ad
[INFO][2023-09-19 23:21:12,511][dance][_maybe_preprocess] Preprocessing done.
[INFO][2023-09-19 23:21:12,789][dance][set_config_from_dict] Setti

epoch:  1
training (sum of 4 losses): 0.4841890081522926
validation (prediction loss): 0.1272758199272512
epoch:  2
training (sum of 4 losses): 0.4898534313004504
validation (prediction loss): 0.12731385463944095
epoch:  3
training (sum of 4 losses): 0.43220654875040054
validation (prediction loss): 0.12729757289264496
epoch:  4
training (sum of 4 losses): 0.41473497446346025
validation (prediction loss): 0.1257743414340414
epoch:  5
training (sum of 4 losses): 0.40186044874210514
validation (prediction loss): 0.12676716495841353
epoch:  6
training (sum of 4 losses): 0.4007334979551454
validation (prediction loss): 0.12180501143531533
epoch:  7
training (sum of 4 losses): 0.4002666192589909
validation (prediction loss): 0.11913859328047662
epoch:  8
training (sum of 4 losses): 0.40463095871351096
validation (prediction loss): 0.11995115804506373
epoch:  9
training (sum of 4 losses): 0.3887418782118187
validation (prediction loss): 0.12354993457400218
epoch:  10
training (sum of 4 losse

[INFO][2023-09-19 23:21:48,355][dance][set_seed] Setting global random seed to 2012473794
[INFO][2023-09-19 23:21:48,357][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/5k_pbmc/5k_pbmc.5kanti_dataset.output_train_mod1.h5ad
[INFO][2023-09-19 23:21:52,351][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/5k_pbmc/5k_pbmc.5kanti_dataset.output_train_mod2.h5ad
[INFO][2023-09-19 23:21:56,280][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/5k_pbmc/5k_pbmc.5kanti_dataset.output_test_mod1.h5ad
[INFO][2023-09-19 23:21:57,844][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/5k_pbmc/5k_pbmc.5kanti_dataset.output_test_mod2.h5ad
[INFO][2023-09-19 23:22:00,448][dance][_maybe_preprocess] Preprocessing done.
[INFO][2023-09-19 23:22:20,576][dance][set_config_from_dict] Setting config 'feature_mod' to 'mod1'
[INFO]

epoch:  1
training (sum of 4 losses): 3.157609720238715
validation (prediction loss): 0.32937001196133775
epoch:  2
training (sum of 4 losses): 3.1117790314439002
validation (prediction loss): 0.32937001196133775
epoch:  3
training (sum of 4 losses): 3.1295425438782782
validation (prediction loss): 0.32937001196133775
epoch:  4
training (sum of 4 losses): 3.1211463090092426
validation (prediction loss): 0.32937001196133775
epoch:  5
training (sum of 4 losses): 3.1010333395673224
validation (prediction loss): 0.32937001196133775
epoch:  6
training (sum of 4 losses): 3.101848100307047
validation (prediction loss): 0.32937001196133775
epoch:  7
training (sum of 4 losses): 3.114398209958948
validation (prediction loss): 0.32937001196133775
epoch:  8
training (sum of 4 losses): 3.097988542740143
validation (prediction loss): 0.32937001196133775
epoch:  9
training (sum of 4 losses): 2.424923767751303
validation (prediction loss): 0.32937001196133775
epoch:  10
training (sum of 4 losses): 3.1

[INFO][2023-09-19 23:38:27,441][dance][set_seed] Setting global random seed to 2012473794
[INFO][2023-09-19 23:38:27,444][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/pbmc_cite/pbmc_cite.citeanti_dataset.output_train_mod1.h5ad


...

[INFO][2023-09-19 23:49:18,546][dance][set_seed] Setting global random seed to 2012473794
[INFO][2023-09-19 23:49:18,548][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/GSE140203_LUNG_atac2gex/GSE140203_LUNG_atac2gex.GSE140203_dataset.output_train_mod1.h5ad


epoch:  8
training (sum of 4 losses): 0.816318076232384
validation (prediction loss): 0.39798599217930725
Early stopped.
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:1')
0.2940707018495889


[INFO][2023-09-19 23:49:18,829][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/GSE140203_LUNG_atac2gex/GSE140203_LUNG_atac2gex.GSE140203_dataset.output_train_mod2.h5ad
[INFO][2023-09-19 23:49:18,859][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/GSE140203_LUNG_atac2gex/GSE140203_LUNG_atac2gex.GSE140203_dataset.output_test_mod1.h5ad
[INFO][2023-09-19 23:49:19,089][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/GSE140203_LUNG_atac2gex/GSE140203_LUNG_atac2gex.GSE140203_dataset.output_test_mod2.h5ad
[INFO][2023-09-19 23:49:23,509][dance][_maybe_preprocess] Preprocessing done.
[INFO][2023-09-19 23:49:23,575][dance][set_config_from_dict] Setting config 'feature_mod' to 'mod1'
[INFO][2023-09-19 23:49:23,576][dance][set_config_from_dict] Setting config 'label_mod' to 'mod2'
[INFO][2023-09-19 23:49:23,577][dance][load_data] Raw data loaded:
Data obj

epoch:  1
training (sum of 4 losses): 1.134829030587123
validation (prediction loss): 0.520773175262135
epoch:  2
training (sum of 4 losses): 1.014776518711677
validation (prediction loss): 0.4842661894334035
epoch:  3
training (sum of 4 losses): 0.8993901289426364
validation (prediction loss): 0.47158381538846206
epoch:  4
training (sum of 4 losses): 0.7638375346477215
validation (prediction loss): 0.43420256581435696
epoch:  5
training (sum of 4 losses): 0.6637009978294373
validation (prediction loss): 0.4096705755272832
epoch:  6
training (sum of 4 losses): 0.6254518444721515
validation (prediction loss): 0.4180234623688568
epoch:  7
training (sum of 4 losses): 0.6043760363872235
validation (prediction loss): 0.41630212115258936
epoch:  8
training (sum of 4 losses): 0.6113392664836004
validation (prediction loss): 0.42329231062817996
epoch:  9
training (sum of 4 losses): 0.6194003866269038
validation (prediction loss): 0.42232533396514965
epoch:  10
training (sum of 4 losses): 0.595

'To reproduce BABEL on other samples, please refer to command lines belows:\nGEX to ADT (subset):\npython babel.py --subtask openproblems_bmmc_cite_phase2_rna_subset --device cuda\n\nGEX to ADT:\npython babel.py --subtask openproblems_bmmc_cite_phase2_rna --device cuda\n\nADT to GEX:\npython babel.py --subtask openproblems_bmmc_cite_phase2_mod2 --device cuda\n\nGEX to ATAC:\npython babel.py --subtask openproblems_bmmc_multiome_phase2_rna --device cuda\n\nATAC to GEX:\npython babel.py --subtask openproblems_bmmc_multiome_phase2_mod2 --device cuda\n'

In [4]:
BabelWrapper_scores

[0.12630768809385384,
 19.13239858295633,
 306.318525496745,
 56.08339258486531,
 3.071854609347063,
 0.10067879455406047,
 0.13530324731383325,
 0.12095927262412384,
 0.6086696308120565,
 0.44328204921667846,
 0.7610827798822227,
 0.5420372746589952,
 0.2385461304314986,
 0.1761410369018866,
 0.2940707018495889,
 0.2527138514491517]

In [5]:
"""Main functionality for starting training.

This code is based on https://github.com/NVlabs/MUNIT.

"""
import argparse
import os
import random

import torch
from sklearn import preprocessing

from dance.datasets.multimodality import ModalityPredictionDataset
from dance.modules.multi_modality.predict_modality.cmae import CMAE
from dance.utils import set_seed


rndseed = random.randint(0, 2147483647)
parser = argparse.ArgumentParser()
parser.add_argument("--output_path", type=str, default="./predict_modality/output", help="outputs path")
parser.add_argument("--resume", action="store_true")
parser.add_argument("-t", "--subtask", default="openproblems_bmmc_cite_phase2_rna")
parser.add_argument("-device", "--device", default="cuda:1")
parser.add_argument("-cpu", "--cpus", default=1, type=int)
parser.add_argument("-seed", "--rnd_seed", default=rndseed, type=int)
parser.add_argument("--span", default=0.3, type=float)
parser.add_argument("--selection_threshold", default=10000, type=int)
parser.add_argument("--max_epochs", default=5, type=int, help="maximum number of training epochs")
parser.add_argument("--batch_size", default=64, type=int, help="batch size")
parser.add_argument("--log_data", default=True, type=bool, help="take a log1p of the data as input")
parser.add_argument("--feature_filter", default=False, type=bool)
parser.add_argument("--normalize_data", default=True, type=bool,
                    help="normalize the data (after the log, if applicable)")
parser.add_argument("--weight_decay", default=1e-4, type=float, help="weight decay")
parser.add_argument("--beta1", default=0.5, type=float, help="Adam parameter")
parser.add_argument("--beta2", default=0.999, type=float, help="Adam parameter")
parser.add_argument("--init", default="kaiming", type=str,
                    help="initialization [gaussian/kaiming/xavier/orthogonal]")
parser.add_argument("--lr", default=1e-4, type=float, help="initial learning rate")
parser.add_argument("--lr_policy", default="step", type=str, help="learning rate scheduler")
parser.add_argument("--step_size", default=100000, type=int, help="how often to decay learning rate")
parser.add_argument("--gamma", default=0.5, type=float, help="how much to decay learning rate")
parser.add_argument("--gan_w", default=10, type=int, help="weight of adversarial loss")
parser.add_argument("--recon_x_w", default=10, type=int, help="weight of image reconstruction loss")
parser.add_argument("--recon_h_w", default=0, type=int, help="weight of hidden reconstruction loss")
parser.add_argument("--recon_kl_w", default=0, type=int, help="weight of KL loss for reconstruction")
parser.add_argument("--supervise", default=1, type=float, help="fraction to supervise")
parser.add_argument("--super_w", default=0.1, type=float, help="weight of supervision loss")
CMAE_scores=[]
for dataset in datasets:
    try:
        opts = parser.parse_args(['--subtask',dataset,'--device',"cuda:2","--span","1.0",'--selection_threshold','10000'])
        device = opts.device

        torch.set_num_threads(opts.cpus)
        rndseed = opts.rnd_seed
        set_seed(rndseed)
        dataset = ModalityPredictionDataset(opts.subtask, preprocess="feature_selection",span=opts.span)
        data = dataset.load_data()

        output_directory = os.path.join(opts.output_path, "outputs")
        checkpoint_directory = os.path.join(output_directory, "checkpoints")
        os.makedirs(checkpoint_directory, exist_ok=True)

        # Manually set batch features
        le = preprocessing.LabelEncoder()
        batch = le.fit_transform(data.data.mod["mod1"].obs["batch"])
        data.data.mod["mod1"].obsm["batch"] = batch

        data.set_config(feature_mod=["mod1", "mod1"], label_mod="mod2", feature_channel_type=[None, "obsm"],
                        feature_channel=[None, "batch"], overwrite=True)

        # Obtain training and testing data
        (x_train, batch), y_train = data.get_train_data(return_type="torch")
        (x_test, _), y_test = data.get_test_data(return_type="torch")
        batch = batch.long().to(device)[:train_leng]
        x_train = x_train.float().to(device)[:train_leng]
        y_train = y_train.float().to(device)[:train_leng]
        x_test = x_test.float().to(device)[:test_leng]
        y_test = y_test.float().to(device)[:test_leng]

        config = vars(opts)
        # Some Fixed Settings
        config["input_dim_a"] = x_train.shape[1]
        config["input_dim_b"] = y_train.shape[1]
        config["resume"] = opts.resume
        config["num_of_classes"] = max(batch) + 1
        config["shared_layer"] = True
        config["gen"] = {
            "dim": 100,  # hidden layer
            "latent": 50,  # latent layer size
            "activ": "relu",
        }  # activation function [relu/lrelu/prelu/selu/tanh]
        config["dis"] = {
            "dim": 100,
            "norm": None,  # normalization layer [none/bn/in/ln]
            "activ": "lrelu",  # activation function [relu/lrelu/prelu/selu/tanh]
            "gan_type": "lsgan",
        }  # GAN loss [lsgan/nsgan]

        model = CMAE(config)
        model.to(device)

        model.fit(x_train, y_train, batch, checkpoint_directory)
        print(model.predict(x_test))
        score=model.score(x_test, y_test)
        print(score)
    except (Exception, BaseException) as e:
        v=e
    else:
        v=score
    finally:
        print(v)
        CMAE_scores.append(v)
        torch.cuda.empty_cache()
"""To reproduce CMAE on other samples, please refer to command lines belows:
GEX to ADT (subset):
python cmae.py --subtask openproblems_bmmc_cite_phase2_rna_subset --device cuda

GEX to ADT:
python cmae.py --subtask openproblems_bmmc_cite_phase2_rna --device cuda

ADT to GEX:
python cmae.py --subtask openproblems_bmmc_cite_phase2_mod2 --device cuda

GEX to ATAC:
python cmae.py --subtask openproblems_bmmc_multiome_phase2_rna --device cuda

ATAC to GEX:
python cmae.py --subtask openproblems_bmmc_multiome_phase2_mod2 --device cuda
"""


[INFO][2023-09-19 23:49:26,070][dance][set_seed] Setting global random seed to 1764036444
[INFO][2023-09-19 23:49:26,071][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/10k_pbmc/10k_pbmc.10kanti_dataset_subset.output_train_mod1.h5ad
[INFO][2023-09-19 23:49:26,145][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/10k_pbmc/10k_pbmc.10kanti_dataset_subset.output_train_mod2.h5ad
[INFO][2023-09-19 23:49:26,318][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/10k_pbmc/10k_pbmc.10kanti_dataset_subset.output_test_mod1.h5ad
[INFO][2023-09-19 23:49:26,372][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/10k_pbmc/10k_pbmc.10kanti_dataset_subset.output_test_mod2.h5ad
[INFO][2023-09-19 23:49:26,653][dance][_maybe_preprocess] Preprocessing done.
[INFO][2023-09-19 23:49:26,925][dance][set_config_from_dict] Setti

Iteration:  0
RMSE Loss: 0.1886800053426272
Iteration:  1
RMSE Loss: 1.5947494365035129
Iteration:  2
RMSE Loss: 0.9046315506943969
Iteration:  3
RMSE Loss: 0.44191695519978075
Iteration:  4


[INFO][2023-09-19 23:49:55,901][dance][set_seed] Setting global random seed to 1764036444
[INFO][2023-09-19 23:49:55,903][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/5k_pbmc/5k_pbmc.5kanti_dataset.output_train_mod1.h5ad


RMSE Loss: 0.2103147143265804
Finish training
tensor([[-0.0113,  0.7089, -0.0164,  ...,  0.3291,  0.2471,  0.1296],
        [-0.0095,  0.6018, -0.0194,  ...,  0.2253,  0.2151,  0.0756],
        [-0.0120,  0.7203, -0.0171,  ...,  0.3288,  0.2328,  0.1287],
        ...,
        [-0.0084,  0.7134, -0.0194,  ...,  0.2996,  0.2353,  0.1207],
        [-0.0113,  0.7348, -0.0183,  ...,  0.3245,  0.2444,  0.1200],
        [-0.0133,  0.9415, -0.0344,  ...,  0.3634,  0.2676,  0.1221]],
       device='cuda:2')
0.2277729976612516
0.2277729976612516


[INFO][2023-09-19 23:50:00,100][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/5k_pbmc/5k_pbmc.5kanti_dataset.output_train_mod2.h5ad
[INFO][2023-09-19 23:50:04,173][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/5k_pbmc/5k_pbmc.5kanti_dataset.output_test_mod1.h5ad
[INFO][2023-09-19 23:50:05,840][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/5k_pbmc/5k_pbmc.5kanti_dataset.output_test_mod2.h5ad
[INFO][2023-09-19 23:50:08,538][dance][_maybe_preprocess] Preprocessing done.
[INFO][2023-09-19 23:50:28,850][dance][set_config_from_dict] Setting config 'feature_mod' to 'mod1'
[INFO][2023-09-19 23:50:28,852][dance][set_config_from_dict] Setting config 'label_mod' to 'mod2'
[INFO][2023-09-19 23:50:28,853][dance][load_data] Raw data loaded:
Data object that wraps (.data):
MuData object with n_obs × n_vars = 6794880 × 10032
  uns:	'dance_config'
  2 mod

...

[INFO][2023-09-20 00:00:37,387][dance][set_seed] Setting global random seed to 1764036444
[INFO][2023-09-20 00:00:37,389][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/GSE140203_LUNG_atac2gex/GSE140203_LUNG_atac2gex.GSE140203_dataset.output_train_mod1.h5ad


RMSE Loss: 1.035069530381413
Finish training
tensor([[-2.8572e-02, -2.0851e-02, -3.0620e-02,  ...,  2.4197e-01,
         -7.5289e-03,  1.9429e-02],
        [-4.4796e-02, -3.3976e-02, -6.4428e-02,  ...,  4.9424e-01,
         -6.7376e-02,  1.7218e-01],
        [-3.1113e-02, -6.2549e-02, -6.6801e-02,  ...,  4.8357e-01,
         -2.5585e-02,  4.4388e-02],
        ...,
        [-1.1424e-02, -7.3146e-03, -1.3566e-02,  ...,  3.6280e-02,
         -2.2445e-03, -3.5800e-04],
        [-9.8991e-03, -4.1649e-03, -1.2124e-02,  ...,  2.7431e-02,
         -3.1868e-03,  2.6997e-03],
        [-1.5027e-02, -6.8820e-03, -2.0316e-02,  ...,  4.4421e-02,
         -5.5963e-03, -3.7436e-04]], device='cuda:2')
0.40812492534175265
0.40812492534175265


[INFO][2023-09-20 00:00:37,697][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/GSE140203_LUNG_atac2gex/GSE140203_LUNG_atac2gex.GSE140203_dataset.output_train_mod2.h5ad
[INFO][2023-09-20 00:00:37,740][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/GSE140203_LUNG_atac2gex/GSE140203_LUNG_atac2gex.GSE140203_dataset.output_test_mod1.h5ad
[INFO][2023-09-20 00:00:38,003][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/GSE140203_LUNG_atac2gex/GSE140203_LUNG_atac2gex.GSE140203_dataset.output_test_mod2.h5ad
[INFO][2023-09-20 00:00:43,917][dance][_maybe_preprocess] Preprocessing done.
[INFO][2023-09-20 00:00:43,995][dance][set_config_from_dict] Setting config 'feature_mod' to 'mod1'
[INFO][2023-09-20 00:00:43,997][dance][set_config_from_dict] Setting config 'label_mod' to 'mod2'
[INFO][2023-09-20 00:00:43,998][dance][load_data] Raw data loaded:
Data obj

Iteration:  0
RMSE Loss: 0.44133008451697
Iteration:  1
RMSE Loss: 0.4408965924088275
Iteration:  2
RMSE Loss: 0.440853515510348
Iteration:  3
RMSE Loss: 0.4408915058698641
Iteration:  4
RMSE Loss: 0.44173318014198854
Finish training
tensor([[-1.0136e-03, -4.2408e-04, -7.6984e-04,  ..., -8.9200e-04,
         -2.7981e-04, -9.7284e-04],
        [ 1.3672e-01, -3.7763e-02, -5.2666e-03,  ..., -4.9064e-03,
         -1.1956e-02, -9.3027e-03],
        [ 4.5742e-02, -1.5399e-02, -5.6545e-03,  ...,  3.5208e-03,
         -7.6927e-03, -1.0795e-02],
        ...,
        [ 5.9517e-02, -1.8111e-02, -3.9698e-03,  ...,  2.3288e-03,
         -8.4486e-03, -1.0019e-02],
        [-1.4758e-05, -1.7102e-03, -2.2090e-03,  ..., -1.0920e-03,
         -6.0191e-04, -1.6234e-03],
        [ 2.7313e-03, -3.4146e-03, -3.9612e-04,  ...,  1.7161e-03,
         -1.3433e-03, -1.4179e-03]], device='cuda:2')
0.43199402164146095
0.43199402164146095


'To reproduce CMAE on other samples, please refer to command lines belows:\nGEX to ADT (subset):\npython cmae.py --subtask openproblems_bmmc_cite_phase2_rna_subset --device cuda\n\nGEX to ADT:\npython cmae.py --subtask openproblems_bmmc_cite_phase2_rna --device cuda\n\nADT to GEX:\npython cmae.py --subtask openproblems_bmmc_cite_phase2_mod2 --device cuda\n\nGEX to ATAC:\npython cmae.py --subtask openproblems_bmmc_multiome_phase2_rna --device cuda\n\nATAC to GEX:\npython cmae.py --subtask openproblems_bmmc_multiome_phase2_mod2 --device cuda\n'

In [6]:
CMAE_scores

[0.2277729976612516,
 torch.cuda.OutOfMemoryError('CUDA out of memory. Tried to allocate 177.19 GiB (GPU 2; 23.70 GiB total capacity; 3.81 GiB already allocated; 19.41 GiB free; 3.97 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF'),
 532.0849321302004,
 96.15730621779346,
 6.606422143459927,
 torch.cuda.OutOfMemoryError('CUDA out of memory. Tried to allocate 980.00 MiB (GPU 2; 23.70 GiB total capacity; 20.48 GiB already allocated; 768.81 MiB free; 22.63 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF'),
 torch.cuda.OutOfMemoryError('CUDA out of memory. Tried to allocate 724.00 MiB (GPU 2; 23.70 GiB total capacity; 20.99 GiB already allocated; 700.81 MiB free; 22.70 GiB reserved in total b

In [7]:
import argparse
import random

import torch

from dance.datasets.multimodality import ModalityPredictionDataset
from dance.modules.multi_modality.predict_modality.scmm import MMVAE
from dance.utils import set_seed


rndseed = random.randint(0, 2147483647)
parser = argparse.ArgumentParser()
parser.add_argument("--output_path", type=str, default="./predict_modality/output", help="outputs path")
parser.add_argument("--resume", action="store_true")
parser.add_argument("-t", "--subtask", default="openproblems_bmmc_cite_phase2_rna")
parser.add_argument("-device", "--device", default="cuda")
parser.add_argument("-cpu", "--cpus", default=1, type=int)
parser.add_argument("-seed", "--rnd_seed", default=rndseed, type=int)

parser.add_argument("--experiment", type=str, default="test", metavar="E", help="experiment name")
parser.add_argument("--obj", type=str, default="m_elbo_naive_warmup", metavar="O",
                    help="objective to use (default: elbo)")
parser.add_argument(
    "--llik_scaling", type=float, default=1., help="likelihood scaling for cub images/svhn modality when running in"
    "multimodal setting, set as 0 to use default value")
parser.add_argument("--batch_size", type=int, default=64, metavar="N", help="batch size for data (default: 256)")
parser.add_argument("--epochs", type=int, default=5, metavar="E", help="number of epochs to train (default: 100)")
parser.add_argument("--lr", type=float, default=1e-4, metavar="L", help="learning rate (default: 1e-3)")
parser.add_argument("--latent_dim", type=int, default=10, metavar="L", help="latent dimensionality (default: 20)")
parser.add_argument("--num_hidden_layers", type=int, default=2, metavar="H",
                    help="number of hidden layers in enc and dec (default: 2)")
parser.add_argument("--r_hidden_dim", type=int, default=100, help="number of hidden units in enc/dec for gene")
parser.add_argument("--p_hidden_dim", type=int, default=20,
                    help="number of hidden units in enc/dec for protein/peak")
parser.add_argument("--pre_trained", type=str, default="",
                    help="path to pre-trained model (train from scratch if empty)")
parser.add_argument("--learn_prior", type=bool, default=True, help="learn model prior parameters")
parser.add_argument("--print_freq", type=int, default=0, metavar="f",
                    help="frequency with which to print stats (default: 0)")
parser.add_argument("--deterministic_warmup", type=int, default=50, metavar="W", help="deterministic warmup")
parser.add_argument("--span", default=0.3, type=float)
MMVAE_scores=[]
for dataset in datasets:
    try:
        args = parser.parse_args(['--subtask',dataset,'--device','cuda:5','--span','1.0'])
        if args.subtask in ['pbmc_cite','openproblems_2022_cite_gex2adt']:
            model_class="rna-protein"
        elif args.subtask in ['GSE117089_A549_gex2atac','GSE117089_sciCAR_gex2atac','GSE127064_AdBrain_gex2atac','GSE127064_p0Brain_gex2atac']:
            model_class="rna-dna"
        else:
            raise RuntimeError("subtask does not exist")
        torch.set_num_threads(args.cpus)
        rndseed = args.rnd_seed
        set_seed(rndseed)
        dataset = ModalityPredictionDataset(args.subtask, preprocess="feature_selection",span=args.span)
        data = dataset.load_data()

        data.set_config(feature_mod="mod1", label_mod="mod2", feature_channel_type="layers", feature_channel="counts",
                        label_channel_type="layers", label_channel="counts")

        # Obtain training and testing data
        x_train, y_train = data.get_train_data(return_type="torch")
        x_test, y_test = data.get_test_data(return_type="torch")
        x_train,y_train=x_train[:train_leng],y_train[:train_leng]
        x_test, y_test = x_test[:test_leng],y_test[:test_leng]
        args.r_dim = x_train.shape[1]
        args.p_dim = y_train.shape[1]
      
        model_class = "rna-protein" if args.subtask == "openproblems_bmmc_cite_phase2_rna" else "rna-dna"
        model = MMVAE(model_class, args).to(args.device)

        model.fit(x_train, y_train)
        print(model.predict(x_test))
        score=model.score(x_test, y_test)
        print(score)
    except (Exception, BaseException) as e:
        v=e
    else:
        v=score
    finally:
        print(v)
        MMVAE_scores.append(v)
        torch.cuda.empty_cache()
"""To reproduce scMM on other samples, please refer to command lines belows:
GEX to ADT (subset):
python scmm.py --subtask openproblems_bmmc_cite_phase2_rna_subset --device cuda

GEX to ADT:
python scmm.py --subtask openproblems_bmmc_cite_phase2_rna --device cuda

ADT to GEX:
python scmm.py --subtask openproblems_bmmc_cite_phase2_mod2 --device cuda

GEX to ATAC:
python scmm.py --subtask openproblems_bmmc_multiome_phase2_rna --device cuda

ATAC to GEX:
python scmm.py --subtask openproblems_bmmc_multiome_phase2_mod2 --device cuda
"""


[INFO][2023-09-20 00:00:46,634][dance][set_seed] Setting global random seed to 631693623
[INFO][2023-09-20 00:00:46,636][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/pbmc_cite/pbmc_cite.citeanti_dataset.output_train_mod1.h5ad


subtask does not exist
subtask does not exist


[INFO][2023-09-20 00:00:47,008][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/pbmc_cite/pbmc_cite.citeanti_dataset.output_train_mod2.h5ad
[INFO][2023-09-20 00:00:47,043][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/pbmc_cite/pbmc_cite.citeanti_dataset.output_test_mod1.h5ad
[INFO][2023-09-20 00:00:47,189][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/pbmc_cite/pbmc_cite.citeanti_dataset.output_test_mod2.h5ad
[INFO][2023-09-20 00:00:48,502][dance][_maybe_preprocess] Preprocessing done.
/home/zyxing/anaconda3/envs/dance/lib/python3.8/site-packages/mudata/_core/mudata.py:491: UserWarning: Cannot join columns with the same name because var_names are intersecting.
  warnings.warn(
/home/zyxing/anaconda3/envs/dance/lib/python3.8/site-packages/mudata/_core/mudata.py:1059: UserWarning: Modality names will be prepended to var_names since there are

====> Epoch: 001 Train loss: 5204.8610
====>             Valid loss: 5037.6015
====> Epoch: 002 Train loss: 4573.8848
====>             Valid loss: 4453.5370
====> Epoch: 003 Train loss: 4092.6883
====>             Valid loss: 5328.1011
====> Epoch: 004 Train loss: 3712.2404
====>             Valid loss: 3651.8699
====> Epoch: 005 Train loss: 3417.9821
====>             Valid loss: 3395.3335


[INFO][2023-09-20 00:01:05,580][dance][set_seed] Setting global random seed to 631693623
[INFO][2023-09-20 00:01:05,582][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/openproblems_2022_cite_gex2adt/openproblems_2022_cite_gex2adt.open_dataset.output_train_mod1.h5ad


...

[INFO][2023-09-20 00:02:58,628][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/GSE117089_sciCAR_gex2atac/GSE117089_sciCAR_gex2atac.GSE117089_dataset.output_test_mod1.h5ad
[INFO][2023-09-20 00:02:58,787][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/GSE117089_sciCAR_gex2atac/GSE117089_sciCAR_gex2atac.GSE117089_dataset.output_test_mod2.h5ad
[INFO][2023-09-20 00:02:59,367][dance][_maybe_preprocess] Preprocessing done.
[INFO][2023-09-20 00:02:59,825][dance][set_config_from_dict] Setting config 'feature_mod' to 'mod1'
[INFO][2023-09-20 00:02:59,827][dance][set_config_from_dict] Setting config 'label_mod' to 'mod2'
[INFO][2023-09-20 00:02:59,828][dance][load_data] Raw data loaded:
Data object that wraps (.data):
MuData object with n_obs × n_vars = 4825 × 199603
  uns:	'dance_config'
  2 modalities
    mod1:	4825 x 10000
      obs:	'cell_name', 'experiment', 'treatment_time', 'batch'
      layer

Expected parameter loc (Tensor of shape (64, 10)) of distribution Laplace(loc: torch.Size([64, 10]), scale: torch.Size([64, 10])) to satisfy the constraint Real(), but found invalid values:
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, n

[INFO][2023-09-20 00:03:08,432][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/GSE127064_p0Brain_gex2atac/GSE127064_p0Brain_gex2atac.GSE126074_dataset.output_test_mod1.h5ad
[INFO][2023-09-20 00:03:08,471][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/GSE127064_p0Brain_gex2atac/GSE127064_p0Brain_gex2atac.GSE126074_dataset.output_test_mod2.h5ad
[INFO][2023-09-20 00:03:08,848][dance][_maybe_preprocess] Preprocessing done.
[INFO][2023-09-20 00:03:09,769][dance][set_config_from_dict] Setting config 'feature_mod' to 'mod1'
[INFO][2023-09-20 00:03:09,771][dance][set_config_from_dict] Setting config 'label_mod' to 'mod2'
[INFO][2023-09-20 00:03:09,772][dance][load_data] Raw data loaded:
Data object that wraps (.data):
MuData object with n_obs × n_vars = 5081 × 239429
  uns:	'dance_config'
  2 modalities
    mod1:	5081 x 10000
      obs:	'batch'
      layers:	'counts'
    mod2:	5081 x 229429
     

====> Epoch: 001 Train loss: 30876.6852
====>             Valid loss: 30327.5222
====> Epoch: 002 Train loss: 29129.3946
====>             Valid loss: 28629.2429
====> Epoch: 003 Train loss: 27639.1652
====>             Valid loss: 27282.0061
====> Epoch: 004 Train loss: 26295.3588
====>             Valid loss: 25943.2981
====> Epoch: 005 Train loss: 25049.5361
====>             Valid loss: 24833.4261
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 1.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]], device='cuda:5')
0.2697611256966332
0.2697611256966332
subtask does not exist
subtask does not exist
subtask does not exist
subtask does not exist
subtask does not exist
subtask does not exist
subtask does not exist
subtask does not exist


'To reproduce scMM on other samples, please refer to command lines belows:\nGEX to ADT (subset):\npython scmm.py --subtask openproblems_bmmc_cite_phase2_rna_subset --device cuda\n\nGEX to ADT:\npython scmm.py --subtask openproblems_bmmc_cite_phase2_rna --device cuda\n\nADT to GEX:\npython scmm.py --subtask openproblems_bmmc_cite_phase2_mod2 --device cuda\n\nGEX to ATAC:\npython scmm.py --subtask openproblems_bmmc_multiome_phase2_rna --device cuda\n\nATAC to GEX:\npython scmm.py --subtask openproblems_bmmc_multiome_phase2_mod2 --device cuda\n'

In [8]:
MMVAE_scores

[RuntimeError('subtask does not exist'),
 RuntimeError('subtask does not exist'),
 0.838180562964654,
 RuntimeError('subtask does not exist'),
 0.5444701041394332,
 0.23805766481510268,
 ValueError("Expected parameter loc (Tensor of shape (64, 10)) of distribution Laplace(loc: torch.Size([64, 10]), scale: torch.Size([64, 10])) to satisfy the constraint Real(), but found invalid values:\ntensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],\n        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],\n        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],\n        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],\n        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],\n        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],\n        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],\n        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],\n        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],\n        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],\n

In [9]:
train_leng=2000
test_leng=700
import argparse
import os
from argparse import Namespace

import anndata
import mudata
import numpy as np
import torch
import scanpy as sc
from dance.data import Data
from dance.datasets.multimodality import ModalityPredictionDataset
from dance.modules.multi_modality.predict_modality.scmogcn import ScMoGCNWrapper
from dance.transforms.cell_feature import BatchFeature
from dance.transforms.graph import ScMoGNNGraph

from dance.utils import set_seed
pipl_scores=[]

def pipeline(inductive=False, verbose=2, logger=None, **kwargs):
    PREFIX = kwargs["prefix"]
    os.makedirs(kwargs["log_folder"], exist_ok=True)
    os.makedirs(kwargs["model_folder"], exist_ok=True)
    os.makedirs(kwargs["result_folder"], exist_ok=True)
    if verbose > 1:
        logger = open(f"{kwargs['log_folder']}/{PREFIX}.log", "w")
        logger.write(str(kwargs) + "\n")

    subtask = kwargs["subtask"]
    cell_filter=True
    dataset = ModalityPredictionDataset(subtask, preprocess=kwargs["preprocessing"])
    # dataset.download_pathway() 测试时需要恢复
    modalities = dataset.load_raw_data()
    if cell_filter:
        for moda in modalities:
            sc.pp.filter_cells(moda,min_genes=3)

    modalities[0]=modalities[0][:train_leng]
    modalities[1]=modalities[1][:train_leng]
    modalities[2]=modalities[2][:test_leng]
    modalities[3]=modalities[3][:test_leng]
    mod1 = anndata.concat((modalities[0], modalities[2]))
    mod2 = anndata.concat((modalities[1], modalities[3]))
    mod1.var_names_make_unique()
    mod2.var_names_make_unique()
    mdata = mudata.MuData({"mod1": mod1, "mod2": mod2})
    train_size = modalities[0].shape[0]
    data = Data(mdata, train_size=train_size)
    data.set_config(feature_mod="mod1", label_mod="mod2")

    data = ScMoGNNGraph(inductive, kwargs["cell_init"], kwargs["pathway"], kwargs["subtask"], kwargs["pathway_weight"],
                        kwargs["pathway_threshold"], kwargs["pathway_path"])(data)
    if not kwargs["no_batch_features"]:
        data = BatchFeature()(data)

    idx = np.random.permutation(modalities[0].shape[0])
    split = {"train": idx[:-int(len(idx) * 0.15)], "valid": idx[-int(len(idx) * 0.15):]}
    kwargs["FEATURE_SIZE"] = modalities[0].shape[1]
    kwargs["TRAIN_SIZE"] = modalities[0].shape[0]
    kwargs["OUTPUT_SIZE"] = modalities[1].shape[1]
    kwargs["CELL_SIZE"] = modalities[0].shape[0] + modalities[2].shape[0]

    if inductive:
        g, gtest = data.data.uns["g"], data.data.uns["gtest"]
    else:
        gtest = g = data.data.uns["g"]

    _, y_train = data.get_train_data(return_type="torch")
    _, y_test = data.get_test_data(return_type="torch")
    y_train=y_train[:train_leng]
    y_test=y_test[:test_leng]
    if not kwargs["no_batch_features"]:
        batch_features = torch.from_numpy(data.data["mod1"].obsm["batch_features"]).float()
        kwargs["BATCH_NUM"] = batch_features.shape[1]
        if inductive:
            g.nodes["cell"].data["bf"] = batch_features[:kwargs["TRAIN_SIZE"]]
            gtest.nodes["cell"].data["bf"] = batch_features
        else:
            g.nodes["cell"].data["bf"] = batch_features

    model = ScMoGCNWrapper(Namespace(**kwargs))

    if kwargs["sampling"]:
        model.fit_with_sampling(g, y_train, split, not inductive, verbose, y_test, logger)
    else:
        model.fit(g, y_train, split, not inductive, verbose, y_test, logger)

    print(model.predict(g, np.arange(kwargs["TRAIN_SIZE"], kwargs["CELL_SIZE"]), device="cpu"))
    score=model.score(g, np.arange(kwargs["TRAIN_SIZE"], kwargs["CELL_SIZE"]), y_test, device="cpu")
    return score



parser = argparse.ArgumentParser()
parser.add_argument("-prefix", "--prefix", default="dance_openproblems_bmmc_atac2rna_test")
parser.add_argument("-t", "--subtask", default="openproblems_bmmc_cite_phase2_rna")
parser.add_argument("-pww", "--pathway_weight", default="pearson", choices=["cos", "one", "pearson"])
parser.add_argument("-pwth", "--pathway_threshold", type=float, default=-1.0)
parser.add_argument("-l", "--log_folder", default="./logs")
parser.add_argument("-m", "--model_folder", default="./models")
parser.add_argument("-r", "--result_folder", default="./results")
parser.add_argument("-e", "--epoch", type=int, default=5)
parser.add_argument("-nbf", "--no_batch_features", action="store_true")
parser.add_argument("-npw", "--pathway", action="store_true")
parser.add_argument("-res", "--residual", default="res_cat", choices=["none", "res_add", "res_cat"])
parser.add_argument("-inres", "--initial_residual", action="store_true")
parser.add_argument("-pwagg", "--pathway_aggregation", default="alpha",
                    choices=["sum", "attention", "two_gate", "one_gate", "alpha", "cat"])
parser.add_argument("-pwalpha", "--pathway_alpha", type=float, default=0.5)
parser.add_argument("-nrc", "--no_readout_concatenate", action="store_true")
parser.add_argument("-bs", "--batch_size", default=1000, type=int)
parser.add_argument("-nm", "--normalization", default="group", choices=["batch", "layer", "group", "none"])
parser.add_argument("-ac", "--activation", default="gelu", choices=["leaky_relu", "relu", "prelu", "gelu"])
parser.add_argument("-em", "--embedding_layers", default=1, type=int, choices=[1, 2, 3])
parser.add_argument("-ro", "--readout_layers", default=1, type=int, choices=[1, 2])
parser.add_argument("-conv", "--conv_layers", default=4, type=int, choices=[1, 2, 3, 4, 5, 6])
parser.add_argument("-agg", "--agg_function", default="mean", choices=["gcn", "mean"])
parser.add_argument("-device", "--device", default="cuda")
parser.add_argument("-sb", "--save_best", action="store_true")
parser.add_argument("-sf", "--save_final", action="store_true")
parser.add_argument("-lr", "--learning_rate", type=float, default=1e-2)
parser.add_argument("-lrd", "--lr_decay", type=float, default=0.99)
parser.add_argument("-wd", "--weight_decay", type=float, default=1e-5)
parser.add_argument("-hid", "--hidden_size", type=int, default=48)
parser.add_argument("-edd", "--edge_dropout", type=float, default=0.3)
parser.add_argument("-mdd", "--model_dropout", type=float, default=0.2)
parser.add_argument("-es", "--early_stopping", type=int, default=0)
parser.add_argument("-c", "--cpu", type=int, default=1)
parser.add_argument("-or", "--output_relu", default="none", choices=["relu", "leaky_relu", "none"])
parser.add_argument("-i", "--inductive", action="store_true")
parser.add_argument("-sa", "--subpath_activation", action="store_true")
parser.add_argument("-ci", "--cell_init", default="none", choices=["none", "svd"])
parser.add_argument("-bas", "--batch_seperation", action="store_true")
parser.add_argument("-pwpath", "--pathway_path", default="./data/h.all.v7.4")
parser.add_argument("-seed", "--random_seed", type=int, default=777)
parser.add_argument("-ws", "--weighted_sum", action="store_true")
parser.add_argument("-samp", "--sampling", action="store_true")
parser.add_argument("-ns", "--node_sampling_rate", type=float, default=0.5)
parser.add_argument("-prep", "--preprocessing", default="none", choices=["none", "feature_selection", "svd"])
parser.add_argument("--span", default=0.3, type=float)
parser.add_argument("-lm", "--low_memory", type=bool, default=True)
for dataset in datasets:
    try:
        args = parser.parse_args(['--subtask',dataset,'--device','cuda:4','--span','1.0'])

        # For test only (low gpu memory setting; to reproduce competition result need >20G GPU memory - v100)
        if args.low_memory:
            print("WARNING: Running in low memory mode, some cli settings maybe overwritten!")
            args.preprocessing = "feature_selection"
            args.pathway = False
            args.sampling = True
            args.batch_size = 10000
            args.epoch = 5
        elif args.subtask == "openproblems_bmmc_multiome_phase2_mod2":
            args.preprocessing = "feature_selection"
        elif args.subtask in ["openproblems_bmmc_cite_phase2_mod2", "openproblems_bmmc_multiome_phase2_rna"]:
            args.sampling = True
            args.edge_dropout = 0

        # Regular settings
        if args.subtask.find("rna") == -1:
            args.pathway = False
        if args.sampling:
            args.pathway = False

        set_seed(args.random_seed)
        torch.set_num_threads(args.cpu)

        score=pipeline(**vars(args))
    except (Exception, BaseException) as e:
        v=e
    else:
        v=score
    finally:
        print(v)
        pipl_scores.append(v)
        torch.cuda.empty_cache()
"""To reproduce scMoGCN on other samples, please refer to command lines belows:
GEX to ADT (subset):
python scmogcn.py --subtask oopenproblems_bmmc_cite_phase2_rna_subset --device cuda

GEX to ADT:
python scmogcn.py --subtask oopenproblems_bmmc_cite_phase2_rna --device cuda

ADT to GEX:
python scmogcn.py --subtask openproblems_bmmc_cite_phase2_mod2 --device cuda

GEX to ATAC:
python scmogcn.py --subtask openproblems_bmmc_multiome_phase2_rna --device cuda

ATAC to GEX:
python scmogcn.py --subtask openproblems_bmmc_multiome_phase2_mod2 --device cuda
"""


[INFO][2023-09-20 00:03:30,302][dance][set_seed] Setting global random seed to 777
[INFO][2023-09-20 00:03:30,305][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/10k_pbmc/10k_pbmc.10kanti_dataset_subset.output_train_mod1.h5ad
[INFO][2023-09-20 00:03:30,372][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/10k_pbmc/10k_pbmc.10kanti_dataset_subset.output_train_mod2.h5ad


[INFO][2023-09-20 00:03:30,537][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/10k_pbmc/10k_pbmc.10kanti_dataset_subset.output_test_mod1.h5ad
[INFO][2023-09-20 00:03:30,591][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/10k_pbmc/10k_pbmc.10kanti_dataset_subset.output_test_mod2.h5ad
[INFO][2023-09-20 00:03:30,882][dance][set_config_from_dict] Setting config 'feature_mod' to 'mod1'
[INFO][2023-09-20 00:03:30,883][dance][set_config_from_dict] Setting config 'label_mod' to 'mod2'


{1}
epoch 0
training:  0.3652422735818718
valid:  0.3343837090721739
testing:  0.18146741463491922
epoch 1
training:  0.21908154056586285
valid:  0.3254443981713573
testing:  0.16418732708172937
epoch 2
training:  0.18339274644400624
valid:  0.3235610587692363
testing:  0.1603420157151971
epoch 3
training:  0.17604387479917108
valid:  0.32244603280291606
testing:  0.15802941970300863
epoch 4
training:  0.16405354089343968
valid:  0.32201306243133027
testing:  0.15707468314684553
min testing 0.15707468314684553 4
converged testing 4 0.15707468314684553
tensor([[-0.0110,  0.0028,  0.0421,  ..., -0.0146,  0.0016, -0.0467],
        [-0.0255,  0.0254,  0.0294,  ..., -0.0274,  0.0063, -0.0490],
        [-0.0118, -0.0124,  0.0499,  ..., -0.0175,  0.0096, -0.0444],
        ...,
        [-0.0231,  0.0104,  0.0402,  ..., -0.0236,  0.0113, -0.0413],
        [ 0.0145, -0.0022,  0.0727,  ...,  0.0038, -0.0073, -0.0637],
        [-0.0241, -0.0118,  0.0425,  ..., -0.0089,  0.0035, -0.0512]])


[INFO][2023-09-20 00:04:00,000][dance][set_seed] Setting global random seed to 777
[INFO][2023-09-20 00:04:00,002][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/5k_pbmc/5k_pbmc.5kanti_dataset.output_train_mod1.h5ad


0.15707468314684553


...

[INFO][2023-09-20 00:23:03,874][dance][set_seed] Setting global random seed to 777
[INFO][2023-09-20 00:23:03,877][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/GSE140203_LUNG_atac2gex/GSE140203_LUNG_atac2gex.GSE140203_dataset.output_train_mod1.h5ad


0.36306614814169197


[INFO][2023-09-20 00:23:04,081][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/GSE140203_LUNG_atac2gex/GSE140203_LUNG_atac2gex.GSE140203_dataset.output_train_mod2.h5ad
[INFO][2023-09-20 00:23:04,109][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/GSE140203_LUNG_atac2gex/GSE140203_LUNG_atac2gex.GSE140203_dataset.output_test_mod1.h5ad
[INFO][2023-09-20 00:23:04,284][dance][_load_raw_data] Loading /home/zyxing/dance/examples/multi_modality/predict_modality/data/GSE140203_LUNG_atac2gex/GSE140203_LUNG_atac2gex.GSE140203_dataset.output_test_mod2.h5ad
[INFO][2023-09-20 00:23:04,778][dance][set_config_from_dict] Setting config 'feature_mod' to 'mod1'
[INFO][2023-09-20 00:23:04,779][dance][set_config_from_dict] Setting config 'label_mod' to 'mod2'


{'1'}
epoch 0
training:  0.5444085359152574
valid:  0.37279332117214026
testing:  0.412506924195407
epoch 1
training:  0.4364692364182858
valid:  0.3447983858754187
testing:  0.38555855544928275
epoch 2
training:  0.40624604956833593
valid:  0.32723461553330435
testing:  0.3686594981326682
epoch 3
training:  0.3789337364530841
valid:  0.3139440669041807
testing:  0.355803887137536
epoch 4
training:  0.37289347865117484
valid:  0.30092099894188074
testing:  0.3431774052135822
min testing 0.3431774052135822 4
converged testing 4 0.3431774052135822
tensor([[ 0.0561,  0.0615,  0.0304,  ..., -0.0551,  0.1097,  0.0347],
        [ 0.0647,  0.0619,  0.0318,  ..., -0.0470,  0.1047,  0.0352],
        [ 0.0649,  0.0629,  0.0328,  ..., -0.0469,  0.1074,  0.0364],
        ...,
        [ 0.0656,  0.0647,  0.0333,  ..., -0.0475,  0.1062,  0.0369],
        [ 0.0633,  0.0697,  0.0351,  ..., -0.0468,  0.1099,  0.0371],
        [ 0.0658,  0.0683,  0.0361,  ..., -0.0476,  0.1075,  0.0384]])
0.343177405213

'To reproduce scMoGCN on other samples, please refer to command lines belows:\nGEX to ADT (subset):\npython scmogcn.py --subtask oopenproblems_bmmc_cite_phase2_rna_subset --device cuda\n\nGEX to ADT:\npython scmogcn.py --subtask oopenproblems_bmmc_cite_phase2_rna --device cuda\n\nADT to GEX:\npython scmogcn.py --subtask openproblems_bmmc_cite_phase2_mod2 --device cuda\n\nGEX to ATAC:\npython scmogcn.py --subtask openproblems_bmmc_multiome_phase2_rna --device cuda\n\nATAC to GEX:\npython scmogcn.py --subtask openproblems_bmmc_multiome_phase2_mod2 --device cuda\n'

In [10]:
pipl_scores

[0.15707468314684553,
 73.220483867648,
 507.28613104440376,
 118.01698103451045,
 3.143192285775932,
 0.10118102912287279,
 0.14702771466757697,
 0.11743710875915477,
 0.9381809873047207,
 0.4987539918878414,
 0.8322997756836706,
 0.6327871388192386,
 0.31837867023058397,
 0.16395969235096525,
 0.36306614814169197,
 0.3431774052135822]